# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - дать ответы на четыре вопроса:
1. Есть ли зависисмость между наличием детей и возвратом кредита в срок.
2. Есть ли зависимость между семейным положением и возвратом кредита в срок.
3. Есть ли зависимость между уровнем дохода и возвратом крелита в срок.
4. Как разные цели кредита влияют на его возврат в срок.

**Ход исследования**

Данные для анализа можно получить из файла /datasets/data.csv.
О качестве данных ничего не известно, поэтому необходимо провести их предварительный обзор, преобработку (проверка данных на ошибки, пропуски, дубликаты) и исправение имеющихся ошибок, пропусков, дубликатов.
Также необходимо будет провести категоризацию данных для ответа на поставленные вопросы.
Таким образом, исследование пройдет в три этапа:
1. Обзор данных.
2. Предобработка данных.
3. Категоризация данных.
4. Получение ответов на поставленные вопросы.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
# импортирование библиотеки pandas, открытие файла с данными, сохранение в переменную data и вывод на экран первых десяти
# строчек таблицы

import pandas as pd
data = pd.read_csv('....csv')
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
# получим общие данные о таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# подсчет пропусков
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64



**Выводы:**

После того, как прочитали данные из файла и вывели результаты на экран, а также получения общей информации о таблице, можно сделать следующие выводы:
1. В таблице представленно 12 столбцов, тип данных в некоторых столбцах отличается. Встречающиеся типы данных в таблице:
   - float64 - 2 столбца
   - int64 - 5 столбцов
   - object - 5 столбцов
   
   Согласно документации к данным, столбцы характеризуются следующим образом:
   - children - количество детей в семье (тип: int64)
   - days_employed - общий трудовой стаж в днях (тип: float64)
   - dob_years - возраст клиента в годах (тип: int64)
   - education - уровень образования клиента (тип: object)
   - education_id - идентификаор уровня образования (тип: object),
   - family_status - семейное положение (тип: object)
   - family_status_id - идентификатор семейного положения (тип: object),
   - gender - пол клиента (тип: object)
   - income_type - тип занятости (тип: object)
   - debt - имел ли задолженность по возврату кредитов (тип: int)
   - total_income - ежемесячный доход (тип: float)
   - purpose - цель получения кредита (тип: object)
2. По полученным данным можно сделать выводы:
   - названия столбцов указаны без ошибок в так называемом "змеином регистре";
   - по общей информации о данных видно, что в столбцах days_employed и total_income имеются пропуски в данных;
   - в столбце days_employed данные - отрицательные числа, нужно изменить их значения на пложительные, т.к. в результате загрузки данных могли произойти ошибки, которые привели к некоректным данным. Также тип этого столбца нужно изменить с вещественного на целочисленный и поправить пропуски;
   - в столбце education необходимо все данные привести к нижнему регистру;
   - в столбце gender все значения привести к нижнему регистру;
   - в столбце total_income изменить тип всех значений на целочисленный, кроме того в этом столбце присутствуют пропуски в значениях, эти пропуски необходимо обработать.
3. Также были выявлены явные дубликаты, которые необходимо будет обработать.

Рассмотрим более детально каждый столбец, для выявления аномалий.

In [4]:
# отобразим общие данные
display(data.describe())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
# поиск аномалий в столбце children
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


В приведенном выше выводе данных видно, что в столбце children имеется 2 аномалии:
1. количество детей 20 у 76 клиентов, такое, конечно, бывает, что у семьи много детей, но настораживает тот факт, что таких клиентов 76, скорее всего здесь ошибка в данных, человеческий фактор. При внесеннии данных человек мог нажать лишнюю клавишу или клиенты сами написали эту цифру.
2. Есть еще одна аномалия: это количство детей -1 у 47 клиентов. На мой взгляд, здесь есть также человеческий фактор, связанный, напрмер, с тем, что неправильно поняли вопрос о количестве детей и написали не 1, а тире 1, и получилось -1.

Эти данные можно попровить, например -1, заменить на единицу, а с теми у кого 20 детей поступить следующим образом: определить возрастную катерию таких людей и взять медианное значение в выборке по указанному возрасту.

In [6]:
# рассмотрим подробнее столбец children, посмотрим, у каких клиентов по 20 детей
display(data[data['children'] == 20])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Как и ожидалось, это ошибка в данных , потому что встречается возраст клиентов, укоторых 20 детей - 21 год.

Для того, чтобы исправить некорректные значения, применим следующие операции: -1 заменим на 1, а значения 20 заменим медианой для каждой возрастной категории. Такая замена обоснована, так как количество элементов с ошибкой составляет 0,35%, поэтому существенно повлиять на результат исследования не сможет.

In [7]:
# заменим значения -1 на 1 в столбце children, выведем информацию по столбцу children, чтобы убедиться в том, что изменения произошли
data.loc[data['children'] == -1, 'children'] = 1
print(data['children'].value_counts())

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64


Для более корректной замены ошибки в данных, определим медиану по количеству детей в зависимости от возрастной категории

In [8]:
# рассчитаем медианы по столбцу children в зависимости от возраста клиентов в целочисленном виде
median_children_before_25 = int(data.loc[data['dob_years'] <= 25]['children'].median())
median_children_26_30 = int(data.loc[(data['dob_years'] >= 26)&(data['dob_years'] <= 30)]['children'].median())
median_children_31_35 = int(data.loc[(data['dob_years'] >= 31)&(data['dob_years'] <= 35)]['children'].median())
median_children_36_40 = int(data.loc[(data['dob_years'] >= 36)&(data['dob_years'] <= 40)]['children'].median())
median_children_41_45 = int(data.loc[(data['dob_years'] >= 41)&(data['dob_years'] <= 45)]['children'].median())
median_children_46_50 = int(data.loc[(data['dob_years'] >= 46)&(data['dob_years'] <= 50)]['children'].median())
median_children_51_55 = int(data.loc[(data['dob_years'] >= 51)&(data['dob_years'] <= 55)]['children'].median())
median_children_upper_56 = int(data.loc[data['dob_years'] >= 56]['children'].median())

In [9]:
print('Медиана для возраста до 25 лет:', median_children_before_25)
print('Медиана для возраста от 26 до 30 лет:', median_children_26_30)
print('Медиана для возраста от 31 до 35 лет:', median_children_31_35)
print('Медиана для возраста от 36 до 40 лет:', median_children_36_40)
print('Медиана для возрастаот 41 до 45 лет:', median_children_41_45)
print('Медиана для возраста от 46 до 50 лет:', median_children_46_50)
print('Медиана для возраста от 51 до 55 лет:', median_children_51_55)
print('Медиана для возраста старше 60 лет:', median_children_upper_56)

Медиана для возраста до 25 лет: 0
Медиана для возраста от 26 до 30 лет: 0
Медиана для возраста от 31 до 35 лет: 1
Медиана для возраста от 36 до 40 лет: 1
Медиана для возрастаот 41 до 45 лет: 0
Медиана для возраста от 46 до 50 лет: 0
Медиана для возраста от 51 до 55 лет: 0
Медиана для возраста старше 60 лет: 0


Разница в значениях небольшая, поэтому для более корректной замены будем использовать значение медины равно 0, но для периода возрастов: от 31 до 40 лет, некорректные данные заполним медианным значением равным 1.

In [10]:
# замена знчений в столбце children == 20 на медианы
data.loc[(data['children'] == 20)&(data['dob_years'] >= 31)&(data['dob_years'] <= 40), 'children'] = median_children_31_35
data.loc[data['children'] == 20, 'children'] = median_children_before_25
print(data['children'].value_counts())

0    14202
1     4888
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


В итоговом вызове видно, что все аномальные значения в столбце были заменены и таким образом всех клиентов можно назделить на шесть категорий:
- 0 - нет детей
- 1 - один ребенок
- 2 - два ребенка
- 3 - три ребенка
- 4 - четыре ребенка
- 5 - пять детей

In [11]:
# заменим отрицательные значения в столбце days_employed на положительные
data.loc[data['days_employed'] < 0, 'days_employed'] = 0 - data['days_employed']
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Как видим в этом столбце все отрицательные числа были заменены на положительные. Ниже сделаем проверку, чтобы убедиться в том, что отрицательных данных нет.

In [12]:
# проверим отсутствие отрицательных данных
print(data[data['days_employed'] < 0])

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [13]:
# проанализируем столбец dob_years
print(data['dob_years'].value_counts())

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


В представленых выше данных видна одна аномалия:
- у 101 человека возраст указан 0 лет. Возможно эта ошибка была связана с тем, что данная информация не была указана клиентами. По приведенной таблице можно попытаться восстановить примерный диапазон возрастов в указанных строках. Возьмем медианный возраст людей по указанному столбцу и заменим аномальные значения. Доля этой аномалии в общем объеме данных составляет 0,47%, таком образом подобная замена не должна существенным образом повлиять на результаты исследования.

In [14]:
# вычислим медиану по столбцу dob_years и сохраним это значение в переменной dob_years_median в целочисленном виде
dob_years_median = int(data['dob_years'].median())
print('Медианное значение возраста:', dob_years_median)

Медианное значение возраста: 42


In [15]:
# заменим значения 0 в столбце dob_years на dob_years_median и выведем уникальные значения столбца на экран
data.loc[data['dob_years'] == 0, 'dob_years'] = dob_years_median
print(data['dob_years'].value_counts())

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64


В приведенном выше выводе видно, что больше нет клиентов со значением возраста 0.

In [16]:
# проанализируем какие уровни образования клиентов предствалены в таблице
print(data['education'].value_counts())

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64


В приведенных выше данных видны следующие уровнь образования:
1. среднее
2. неоконченное высшее
3. высшее
4. начальное
5.ученая степень

Выявить общее количество клиентов по каждому уровню сложно, так как данные представлены в разных регистрах. Изменим эти данные и сравним полученный столбец со столбцом education_id. Это необходимо для того, чтобы понять, правильно ли указана категория от 0 до 4 для каждого уровня образования.

In [17]:
# приведение всех значений в столбце education к нижнему регистру
data['education'] = data['education'].str.lower()

In [18]:
# выведем на экран значения столбца education и education_id
print('Значения столбца education:')
print(data['education'].value_counts())
print()
print('Значения столбца education_id:')
print(data['education_id'].value_counts())

Значения столбца education:
среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Значения столбца education_id:
1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64


В результате произведенных выше действий повысили читабельность столбца education. Также можно сделать вывод, что категории из колонки education_id указаны верно, и теперь видно для какого уровня образования какая катергория указана:
- 0 - высшее
- 1 - среднее
- 2 - неоконченное высшее
- 3 - начальное
- 4 - ученая степень

Но в будущем хотелось бы, чтобы информация о соответсвии уровня образования его категории указывалась в документации к данным.

In [19]:
# проанализируем столбец family_status
print(data['family_status'].value_counts())

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


В приведенных выше данных видны следующие виды семейного положения:

- женат/замужем
- гражданский брак
- не женат/не замужем
- в разводе
- вдовец/вдова

В полученной таблице видно, что данные представлены в разных регистрах. Изменим эти данные и сравним полученный столбец со столбцом family_status_id. Это необходимо для того, чтобы понять, правильно ли указана категория от 0 до 4 для каждого вида семейного положения.

In [20]:
# приведение всех значений в столбце family_status к нижнему регистру
data['family_status'] = data['family_status'].str.lower()

In [21]:
# выведем на экран значения столбца education и education_id
print('Значения столбца family_status:')
print(data['family_status'].value_counts())
print()
print('Значения столбца family_status_id:')
print(data['family_status_id'].value_counts())

Значения столбца family_status:
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Значения столбца family_status_id:
0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64


В результате произведенных выше действий можно сделать вывод, что категории из колонки family_status_id указаны верно, и теперь видно для какого вида семейного положения какая катергория указана:

- 0 - женат/замужен
- 1 - гражданский брак
- 2 - вдовец/вдова
- 3 - в разводе
- 4 - не женат/не хамужем

Но в будущем хотелось бы, чтобы информация о соответсвии семейного положения его категории указывалась в документации к данным.

In [22]:
# проанализируем столбец gender
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


Данный столбец показал, что в одном из значений указан неверный гендер, скорее всего это случайная ошибка. Также в этом столбце все значения в верхнем регистре, а для анализа удобнее было бы, если бы все значения столбца были в нижнем регистре.
Значение в столбце равное XNA мы исправлять не будем, так как невозможно с точностью определить пол человека и исправить это значение. Поэтому поправим только регистр, приведем все значения к нижнему регистру.

In [23]:
# приведение всех значений в столбце gender к нижнему регистру
data['gender'] = data['gender'].str.lower()
print(data['gender'].value_counts())

f      14236
m       7288
xna        1
Name: gender, dtype: int64


Видим, что данные в столбце изменены. В текущем анализе данных нам этот столбец не пригодится. Но на будущее желательно учтонить данные по тому клиенту, который неверно указал пол.

In [24]:
# проанализируем столбец income_type
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


В данном столбце никаких аномалий и ошибок в данных не обнаружено.

In [25]:
# проанализируем столбец debt
print(data['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


В данном столбце также никаких анамалий не обнаружно. Единтсвенное, в документации к данным не указано, что значит 0 и 1, это количество долгов, или это булевское выражение False для 0 и True для 1.
Так как больше никаких данных о количестве долгов не указано и у нас значения только 0 и 1, будем считать, что это будевские значения, для которых:

- 0 - False
- 1 - True

Но в будущем данный столбец должен иметь комментарий в документации к данным.

In [26]:
# проанализируем столбец total_income. Мы не сможем найти по этому столбцу уникальные значения, так как записей будет много
# но мы можем посмотреть, например, минимальное и максимально значения, для того, чтобы исключить наличие в столбце отрицательных чисел
print('Максимальное значение с столбце total_income: ', data['total_income'].max())
print('Минимальное значение с столбце total_income: ', data['total_income'].min())

Максимальное значение с столбце total_income:  2265604.028722744
Минимальное значение с столбце total_income:  20667.26379327158


По проведенному анализу видно, что отрицательных значений нет, соответсвенно в данном столбце нужно заполнить пропуски и изменить тип данных с вещественного на целочисленный.

In [27]:
# проанализируем столбец purpose
print(data['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

По полученным выше данным видно, что в столбце много однотипных целей получения кредита, но выявить их очень сложно, так как данные отлтчаются склонениями, одна и та же операция описана разными словами. Для того, чтобы привести эти значения к единому виду, необходимо провести лемматизацию и сгруппировать одинаковые цели на основании лемм. 

In [28]:
# выведем итоговый датасет после всех преобразований
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу


В итоговой таблице видно, что все ошибки в значениях (разные регистры, отрицательные числа, аномальные значения) были устранены.

Осталось провести окончательную обработку данных: исправить пропуски, изменить тип данных в столбцах с вещественного на целочисленный, удалить дубликаты, произвести лемматизацию данных в последнем столбце для выделения категорий целей кредита.

## Шаг 2. Предобработка данных

### Обработка пропусков

Для того, чтобы обработать пропуски еще раз посмотрим на таблицу и определим в каких столбцах не хватает данных.

In [29]:
# найдем все строки с пропусками
display(data[data['days_employed'].isna()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,m,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,m,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,не женат / не замужем,4,f,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,f,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,f,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,среднее,1,женат / замужем,0,m,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,f,сотрудник,0,NaN,свадьба
21497,0,NaN,48,высшее,0,женат / замужем,0,f,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,f,сотрудник,0,NaN,строительство жилой недвижимости


По полученной таблице видно, что пропуски в колонках days_employed и total_income совпадают. Это говорит о том, что клиенты, скорее всего сами не указали эти данные. Дозаполним пропуски, использую группировку по признакам и медиану в этих группах по столбцам days_employed и total_income.

Но перед тем как начать заполнять пропуски в значениях необходима дополнительная обработка стобца days_employed. 

Выведем еще раз эту таблицу на экран.

In [30]:
# выведем на экран отсортированнные значения столбца days_employed
display(data[data['days_employed'] > 16920])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,f,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,не женат / не замужем,4,f,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,m,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,f,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,m,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,m,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,m,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,среднее,1,женат / замужем,0,f,пенсионер,0,153864.650328,сделка с автомобилем


В полученном выводе мы увидели следующую аномалию, например, возьмем строку 18: в ней трудовой стаж составляет 400281 день, если перевести это число в годы, из расчета того, что при формировании трудового стажа год принимают равным 360 дням, получаем, что человек перед выходом на пенсию работал: 400281 / 360 = 1111 лет! Такого быть не может.

Значение, взятое в формуле: 16920, формируется следующим образом. Предположим, что человек работал с 18 до 65 лет, итого его трудовой стаж в годах составляет 47 лет (65 - 18). Чтобы представить это значение в днях, умножим число отработанных лет на 360. Получаем: 47  * 360 = 16920.

И таких строк в таблице - 3447 из 21525, что сотавляет около 16% от всех данных. Возможно, такие искажения были получены неправильным пересчетом данных. Это достаточно большое значение, поэтому если будем считать медианное значение даже по категориям, получим существенные искажения в данных.

Попробуем преобразовать эти аномалии. Выясним, для людей какого возраста присуща такая аномалия.

In [31]:
# выведем количество уникальных возрастов, в которых есть указанная аномалия
print(data[data['days_employed'] > 16920]['dob_years'].value_counts())

59    254
60    243
62    235
61    216
57    212
58    208
63    192
56    184
64    179
55    162
54    145
66    139
65    136
67    132
53    105
52     95
68     80
69     74
51     73
50     61
70     54
71     48
49     30
72     28
42     26
48     20
46     13
47     13
45     11
44     10
43      9
38      8
40      7
73      6
41      6
37      5
74      4
39      4
27      3
36      3
34      3
32      3
33      2
26      2
31      1
35      1
28      1
22      1
Name: dob_years, dtype: int64


По полученному выводу видно, что указанная аномалия встречается во всех возрастных категориях. Для более корректного подсчета и заполнения пустых значений в таблице, поправим аномалии. Перезапишем данные этого столбца, для этого возьмем медиану по каждой возрастной категории. Возрастные категории будем делать с шагом 5, первая возрастная категория до 25 лет включительно, и последняя старше 71 года включительно.

In [32]:
# напишем функцию, определяющие медианное знаяение для заданного интервала
def get_median_years_employed(first_volume, second_volume):
    '''
    Определяет медианы значений из указанного интервала значений
    
    param: first_volume - первое значение в интервале включительно
    param: second_volume - второе значение в итнервале включительно
    return: медиана значений из указанного интервала
    '''
    return data.loc[(data['dob_years'] >= first_volume)&(data['dob_years'] <= second_volume)]['days_employed'].median()

In [33]:
# расчет медиан первой и последне категории, а также расчет медиан интервалов с использованием функции
median_before_25 = data.loc[data['dob_years'] <= 25]['days_employed'].median()
median_26_30 = get_median_years_employed(26, 30)
median_31_35 = get_median_years_employed(31, 35)
median_36_40 = get_median_years_employed(36, 40)
median_41_45 = get_median_years_employed(41, 45)
median_46_50 = get_median_years_employed(46, 50)
median_51_55 = get_median_years_employed(51, 55)
median_56_60 = get_median_years_employed(56, 60)
median_61_65 = get_median_years_employed(61, 65)
median_66_70 = get_median_years_employed(66, 70)
median_upper_71 = data.loc[data['dob_years'] >= 71]['days_employed'].median()

In [34]:
# выведем на экран полученные значения медиан
print('Медиана стажа работы в категории до 25 лет:', median_before_25)
print('Медиана стажа работы в категории от 26 до 30 лет:', median_26_30)
print('Медиана стажа работы в категории от 31 до 35 лет:', median_31_35)
print('Медиана стажа работы в категории от 36 до 40 лет:', median_36_40)
print('Медиана стажа работы в категории от 41 до 45 лет:', median_41_45)
print('Медиана стажа работы в категории от 46 до 50 лет:', median_46_50)
print('Медиана стажа работы в категории от 51 до 55 лет:', median_51_55)
print('Медиана стажа работы в категории от 56 до 60 лет:', median_56_60)
print('Медиана стажа работы в категории от 61 до 65 лет:', median_61_65)
print('Медиана стажа работы в категории от 66 до 70 лет:', median_66_70)
print('Медиана стажа работы в категории от 71 года:', median_upper_71)

Медиана стажа работы в категории до 25 лет: 797.5918331374241
Медиана стажа работы в категории от 26 до 30 лет: 1228.638681675905
Медиана стажа работы в категории от 31 до 35 лет: 1496.0708150968378
Медиана стажа работы в категории от 36 до 40 лет: 1812.084853902544
Медиана стажа работы в категории от 41 до 45 лет: 2029.7750653706182
Медиана стажа работы в категории от 46 до 50 лет: 2411.5253843148307
Медиана стажа работы в категории от 51 до 55 лет: 3682.216121806053
Медиана стажа работы в категории от 56 до 60 лет: 336682.4288389096
Медиана стажа работы в категории от 61 до 65 лет: 353424.71246236726
Медиана стажа работы в категории от 66 до 70 лет: 361108.6961660925
Медиана стажа работы в категории от 71 года: 360170.42288407945


В приведенных выше значениях медиан видна одна особенность: начиная с категории от 56 лет и далее медиана составляет от 336682 дней, что говорит о том, что во всех значениях, начиная с 56 летнего возраста в столбце dob_years ошибка. Для указанных интервалов аномальные значения заменять на медианные не будем, так как данные для этих категорий использовать невозможно. Кроме того, необходимо разобраться, почему для этих категорий возникли такие ошибки. Возможно, эта ошибка носит какой-то технический характер. Или неправильный пересчет в данных.

In [35]:
# заменим аномальные значения интервала на значения медианы, для всех интервалов кроме тех, что начинаются с 56 лет
data.loc[(data['dob_years'] <= 25)&(data['days_employed'] > 16920), 'days_employed'] = median_before_25
data.loc[(data['dob_years'] >= 26)&(data['dob_years'] <= 30)&(data['days_employed'] > 16920), 'days_employed'] = median_26_30
data.loc[(data['dob_years'] >= 31)&(data['dob_years'] <= 35)&(data['days_employed'] > 16920), 'days_employed'] = median_31_35
data.loc[(data['dob_years'] >= 36)&(data['dob_years'] <= 40)&(data['days_employed'] > 16920), 'days_employed'] = median_36_40
data.loc[(data['dob_years'] >= 41)&(data['dob_years'] <= 45)&(data['days_employed'] > 16920), 'days_employed'] = median_41_45
data.loc[(data['dob_years'] >= 46)&(data['dob_years'] <= 50)&(data['days_employed'] > 16920), 'days_employed'] = median_46_50
data.loc[(data['dob_years'] >= 51)&(data['dob_years'] <= 55)&(data['days_employed'] > 16920), 'days_employed'] = median_51_55

In [36]:
print(data[data['days_employed'] > 16920]['dob_years'].value_counts())

59    254
60    243
62    235
61    216
57    212
58    208
63    192
56    184
64    179
66    139
65    136
67    132
68     80
69     74
70     54
71     48
72     28
73      6
74      4
Name: dob_years, dtype: int64


В распечатанных выше столбцах видно, что часть аномальных значений мы заменили, остались только аномально высокие значения для возрастов от 56 лет. Эти данные мы оставим без зменений, т.к. нужны исхоные данные. Сложно подобрать и угадать, какие там должны быть корректные данные. Поэтому для того, что окончательно исправить эту аномалию, необходимо завпросить дополнительные данные.

Заполним пропуски в данном столбце, также рассчитав медиану по уже по испраленным значениям. Алгоритм будет такой же, как и выше. Использовать для заполнения интервалов будем функцию get_median_years_employed(), написанную ваше, для интервалов от 56 лет будем рассчитывать только медианы.

In [37]:
# рассчитаем одно из новых значений медианы
median_update_46_50 = get_median_years_employed(46, 50)

In [38]:
# распечатаем одно из значений заново рассчитанных медиан 
print('Медиана стажа работы в категории от 46 до 50 лет:', median_update_46_50)

Медиана стажа работы в категории от 46 до 50 лет: 2411.5253843148307


На этом примере увидели главное преимущество медианы. Даже при наличии аномально высоких значений что до их корректировки, что после, значения медианы совпадают. Поэтому для заполнения пропусков будем использовать уже рассчитанные значения. Заполнение пропусков также будет происходить по возрастным категориям.

In [39]:
data.loc[(data['dob_years'] <= 25)&(data['days_employed'].isna()), 'days_employed'] = median_before_25
data.loc[(data['dob_years'] >= 26)&(data['dob_years'] <= 30)&(data['days_employed'].isna()), 'days_employed'] = median_26_30
data.loc[(data['dob_years'] >= 31)&(data['dob_years'] <= 35)&(data['days_employed'].isna()), 'days_employed'] = median_31_35
data.loc[(data['dob_years'] >= 36)&(data['dob_years'] <= 40)&(data['days_employed'].isna()), 'days_employed'] = median_36_40
data.loc[(data['dob_years'] >= 41)&(data['dob_years'] <= 45)&(data['days_employed'].isna()), 'days_employed'] = median_41_45
data.loc[(data['dob_years'] >= 46)&(data['dob_years'] <= 50)&(data['days_employed'].isna()), 'days_employed'] = median_46_50
data.loc[(data['dob_years'] >= 51)&(data['dob_years'] <= 55)&(data['days_employed'].isna()), 'days_employed'] = median_51_55
data.loc[(data['dob_years'] >= 56)&(data['dob_years'] <= 60)&(data['days_employed'].isna()), 'days_employed'] = median_56_60
data.loc[(data['dob_years'] >= 61)&(data['dob_years'] <= 65)&(data['days_employed'].isna()), 'days_employed'] = median_61_65
data.loc[(data['dob_years'] >= 66)&(data['dob_years'] <= 70)&(data['days_employed'].isna()), 'days_employed'] = median_66_70
data.loc[(data['dob_years'] >= 71)&(data['days_employed'].isna()), 'days_employed'] = median_upper_71

Пропуски в столбце days_employed были заполнены, но аномальные значения для возрастных категорий от 56 лет сохраняются. Корректность указанных данных нужно уточнять, необъодимо увидеть первоначальные данные для коррекного исправления аномалий. В нашем анализе нам эти данные не пригодятся, но если в дальнейшем понадобится анализ в зависимости от стажа работы, то данные для этого восстановены.

Теперь обратимся к столбцу total_income для того, чтобы заполнить пропуски. Из таблицы видно, что все клиенты разбиты по категориям заработка, поэтому для того, чтобы более точно указать доход на месте пропусков, узнаем также медиану по каждой категории. В таком случае мы получим меньше расхождений, чем при расчете среднего значения.

In [40]:
# выведем на экран уникальные значения столбца income_type
print(data['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64


Итак, мы видим категории доходов у клиентов. Определим по каждой категории медиану, для того, чтобы этими значениями заполнить пропуски.

In [41]:
# определим медиану для четырех категорий: сотрудник - employee, компаньон - companion, пенсионер - pensioner, госслужищий - civil_servant
employee = data.loc[data['income_type'] == 'сотрудник']['total_income'].median()
companion = data.loc[data['income_type'] == 'компаньон']['total_income'].median()
pensioner = data.loc[data['income_type'] == 'пенсионер']['total_income'].median()
civil_servant = data.loc[data['income_type'] == 'госслужащий']['total_income'].median()
# по предпринимателями будем брать среднее значение, так как их всего два
businessmen = data.loc[data['income_type'] == 'предприниматель']['total_income'].mean()

In [42]:
# выведем на экран полученные медианы и среднее значение
print('Медиана дохода по сотрудникам:', employee)
print('Медиана дохода по компаньонам:', companion)
print('Медиана дохода по пенсионерам:', pensioner)
print('Медиана дохода по госслужащим:', civil_servant)
print('Среднее значение дохода по предпринимателям:', businessmen)

Медиана дохода по сотрудникам: 142594.39684740017
Медиана дохода по компаньонам: 172357.95096577113
Медиана дохода по пенсионерам: 118514.48641164352
Медиана дохода по госслужащим: 150447.9352830068
Среднее значение дохода по предпринимателям: 499163.1449470857


In [43]:
# заменим пропуски в столбце total_income на медианны по категориям
data.loc[(data['income_type'] == 'сотрудник')&(data['total_income'].isna()), 'total_income'] = employee
data.loc[(data['income_type'] == 'компаньон')&(data['total_income'].isna()), 'total_income'] = companion
data.loc[(data['income_type'] == 'пенсионер')&(data['total_income'].isna()), 'total_income'] = pensioner
data.loc[(data['income_type'] == 'госслужащий')&(data['total_income'].isna()), 'total_income'] = civil_servant
data.loc[(data['income_type'] == 'предприниматель')&(data['total_income'].isna()), 'total_income'] = businessmen

Проверим, все ли пропуски в графе total_income заполнены

In [44]:
# проверка на наличие пропусков
print(data.isna().sum())

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Теперь все пропуски устранены, можно переходить к следующему этапу.

**Вывод**

Пропуски находились в двух столбцах: 
- days_employed
- total_income

Со столбцом total_income не возникло никаких трудностей в заполнении пропусков, всех клиентов можно было разделить по категориям в зависимости от типа заработка (income_type). Было выделено 5 типов заработка по четырем из них мы высчитывали значение медианы и этими значениями заполняли пропуски. Но по одной категории, предприниматель, мы вычислили среднее значение, так как таких клиентов было 2 поэтому значение медианы здесь не информтивно.

Со столбцом days_employed пришлось разбираться дольше, так как в этом столбце было очень много аномальных значение (стаж работы более 900 лет!), часть из этих аномальных значений удалось исправить, путем замены аномалии на медиану, высчитанную в зависимости от возратсной категории. Подсчет медиан мы автоматизировали, путем написания функции. Сделали это для того, чтобы не дублировать код. После этого заменили на медианы и оставшиеся пропуски в данных. Но этот столбец остается под большим вопросом, так как для 2624 строк, почти 12% данных, значения также остаются аномальными. В нашем анализе этот столбец не пригодится, но для дайнейшего анализа необходимо выяснить причину, по которой были указаны неверные данные: человеческий фактор, проблемы с сохранением данных, неверная конвертация данных и т.д.

### Замена типа данных

Заменим тип данных в двух стобцах. Выведем еще раз на экран информацию по данным.

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Столбцы days_employed и total_income тип данных float64, заменим вещественный типа данных на целочисленный.

In [46]:
# заменим типа данных в столбце days_employed
data['days_employed'] = data['days_employed'].astype('int')

In [47]:
# заменим типа данных в столбце total_income
data['total_income'] = data['total_income'].astype('int')

In [48]:
# выведем информацию по data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [49]:
# выведем 5 строчек data, чтобы убедиться в том, что изменения типов произошли
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Как видим выше, данные в столбцах days_employed и total_income стали целочисленными. 

Теперь для дальнейшего удобства работы с таблицей, удалим имеющиеся дубликаты.

### Обработка дубликатов

При первичном анализе таблицы было выявлено, что в нашем датафрейме могут быть явные дубликаты. Выведем еще раз на экран количество дубликатов.

In [50]:
# вывод на экран количества неявных дубликатов
print(data.duplicated().sum())

72


Итого получилось 72 строки явных дубликатов, что составляет около 0,33%. Это значение небольшое, поэтому удаление явных дубликатов никак не должно повляить на итоговые результаты. Поэтому удалим явные дубликаты с удалением старых индексов и формированием новых.

In [51]:
# удаление явных дубликатов (с удалением старыхх индексов и формированием новых)
data = data.drop_duplicates().reset_index(drop=True)

In [52]:
# еще раз выведем на экран количество явных дубликатов
print(data.duplicated().sum())

0


**Вывод**

Все явные дубликаты удалены из датафрейма, старые индексы удалены и сформированы новые. 

Теперь датафрейм полностью готов к проведению анализа: все пропуски заполнены,  данные приведены к одному регистру, изменены типы данных, где это было необходимо, удалены явные дубликаты, все аномалии в значениях, которые можно было поправить, были поправлены.

### Лемматизация

В последнем столбце списка мы видим, что в целом цели кредита соответствуют нескольким категориям, но из-за того, что написаны эти слова по-разному: есть склонения, дополнительные слова, то очень сложно все эти данные свести к нескольким категориям. Попробуем лемматизировать эти данные.

In [53]:
# для лемматизации импортируем модуль 
from pymystem3 import Mystem
m = Mystem()

In [54]:
# импортируем Counter из модуля coLlection для подсчета лемм
from collections import Counter

Проанализируем количество лемм в столбце purpose

In [55]:
# приведем все цели в столбце purpose к их леммам и выведем эти данные на экран
data_join = ','.join(data['purpose'])
lemma = m.lemmatize(data_join)
print(lemma)

['покупка', ' ', 'жилье', ',', 'приобретение', ' ', 'автомобиль', ',', 'покупка', ' ', 'жилье', ',', 'дополнительный', ' ', 'образование', ',', 'сыграть', ' ', 'свадьба', ',', 'покупка', ' ', 'жилье', ',', 'операция', ' ', 'с', ' ', 'жилье', ',', 'образование', ',', 'на', ' ', 'проведение', ' ', 'свадьба', ',', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ',', 'покупка', ' ', 'недвижимость', ',', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ',', 'сыграть', ' ', 'свадьба', ',', 'приобретение', ' ', 'автомобиль', ',', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ',', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ',', 'недвижимость', ',', 'строительство', ' ', 'недвижимость', ',', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ',', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ',', 'недвижимость', ',', 'приобретение', ' ', 'автомобиль', ',', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ',', 'сыграть', ' ', 'свадьба', ',', 'опера

In [56]:
# подсчитаем количество часто повторяющихся лемм, для того, чтобы найти базовые слова для различных названий в столбце purpose
print(Counter(lemma))

Counter({' ': 33568, ',': 21452, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2323, 'свой': 2230, 'на': 2221, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 767, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


Благодаря подсчету количества лемм, можно категоризировать данные в столбце purpose И выделить основные напрвления, на которые берется кредит:
- недвижимость (недвижимость, жилье, жилой)
- автомобиль (автомобиль)
- свадьба (свадьба)
- образование (образование)

Таким образом можно написать функцию для определения категорий по каждой цели получения кредита. Будет реализовано ниже в пункте категоризация данных.

**Вывод**

Благодаря лемматизации процесс объединения схожих данных в целях их категоризации для дальнейшего анализа оказался очень простым.
Нам не нужно писать сложные циклы для перебова все возможных значений. Вместо этого мы привели все слова к их леммам и определили наиболее часто встречающиеся данные. Далее приведем категоризацию данных на основании полученной информации.

### Категоризация данных

Категоризацию данных нам нужно провести по четырем видам данных, т.к. нужно ответить на 4 вопроса:
- категоризируем столбец children
- категоризируем столбец family_status
- категоризируем столбец total_income
- категоризируем столбец purpose

Во время категоризации данных мы будем добавлять столбцы с категориями в датафрейм. Для того, чтобы в дальнейшем можно было использовать эти категории для ответа на поставленные вопросы.

Для удобства анализа разобьем всех клиентов по следующим категориям, в зависимости от количество детей. Общепринятой классификацие является следующая:
- клиенты без детей (количество детей 0)
- клиенты с детьми (количество детей от 1)

Создадим функцию для распределения количества детей по категориям.

In [57]:
# функция для категоризации по количеству детей
def get_category_children(row):
    '''
    Возвращает категорию в зависимости от количества детей
    
    Выделено 2 категории: клиенты без детей (children == 0), клиенты с детьми (все остальные).
    param: row столбец датафрейма data['children']
    '''
    children = row['children']
    if children == 0:
        return 'клиенты без детей'
    else:
        return 'клиенты с детьми'

In [58]:
# применим данную функцию и создадим новый столбец
data['children_category'] = data.apply(get_category_children, axis=1)

In [59]:
# проверим, что в столбце создалась необходимая нам категория
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,клиенты с детьми
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,клиенты с детьми
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,клиенты без детей
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,клиенты с детьми
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,клиенты без детей


Для столбца family_status, на мой взгляд, нужная категоризация данных уже представлена в исходной таблице. Поэтому бедм использовать следующие категории:
- женат/замужем
- не женат/ не замужем
- гражданский брак
- в разводе, 
- вдовец/вдова


Проведем категоризацию данных для столбца total_income.
Доходы можно разделить на три категории:
- высокий уровень дохода
- средний уровень дохода
- низкийуровень дохода

Для более точной классификации клиентов в зависимости от уровня дохода, проведем двухуровневое формирование интервала для клиентов со средним уровнем дохода. Используем для этого одно из свойств медианы: половина из набора элементов не меньше медианы, а вторая половина не больше. Для начала найдем median_general для всего набора значений в total_income, а потом проведем эту операцию еще дважды: для нахождения median_lower набора элементов, находящися до первой median_general, и определим median_upper набора элементов, находящихся после первой median_general. Таким образом, на мой взгяд, мы получим более достоверное распределение доходов клиентов по категориям.

In [60]:
# определим median_general в целочисленном выражении
median_general = int(data['total_income'].median())
print('median_general:', median_general)

median_general: 142594


In [61]:
# определим median_lower в целочисленном выражении
median_lower = int(data.loc[data['total_income'] < median_general]['total_income'].median())
print('median_lower:', median_lower)

median_lower: 103460


In [62]:
# определим median_second_volume в целочисленном выражении
median_upper = int(data.loc[data['total_income'] > median_general]['total_income'].median())
print('median_upper', median_upper)

median_upper 196593


Таким образом, получили необходимые значения и можно проводить категоризацию данных по следующим категориям:
- высокий уровень дохода (total_income >= median_upper)
- средний уровень дохода (median_lower < total_income < mmedian_upper)
- низкий уровень дохода (total_income <= median_lower)

Напишем функция для распределения клиентов по категориям.

In [63]:
# функция для категоризации по уровню дохода
def get_category_total_income(row, median_lower, median_upper):
    '''
    Возвращает категорию в зависимости от уровня дохода
     
    Выделено 3 категории: высокий уровень дохода (total_income >= median_upper), 
    средний уровень дохода (median_first_volume < total_income < median_upper),
    низкий уровень дохода (total_income < median_first_volume)
    param: row столбец датафрейма data['total_income']
    param: median_upper
    param: median_lower
    
    '''    
    total_income = row['total_income']
    if total_income >= median_upper:
        return 'высокий уровень дохода'
    if median_lower < total_income < median_upper:
        return 'средний уровень дохода'
    if total_income <= median_lower:
        return 'низкий уровень дохода'

In [64]:
# применим данную функцию и создадим новый столбец
data['total_income_category'] = data.apply(get_category_total_income, args=(median_lower, median_upper), axis=1)

In [65]:
# проверим, что в столбце создалась необходимая нам категория
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,total_income_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,клиенты с детьми,высокий уровень дохода
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,клиенты с детьми,средний уровень дохода
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,клиенты без детей,средний уровень дохода
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,клиенты с детьми,высокий уровень дохода
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,клиенты без детей,средний уровень дохода


И, наконец, выделем категории для столбца purpose. Благодаря подсчету количества лемм, можно категоризировать данные в столбце purpose:

- недвижимость (недвижимость, жилье, жилой)
- автомобиль (автомобиль)
- свадьба (свадьба)
- образование (образование)

Таким образом можно написать функцию для определения категорий по каждой цели получения кредита.

In [66]:
# напишем функцию для перебора всех лем и определения категорий
def get_category_purpose(row):
    '''
    Возвращает категорию в зависимости от целей кредита
    
    param: row строка из data, столбец purpose
    '''
    lem_purpose = m.lemmatize(row['purpose'])
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose) or ('жилой' in lem_purpose):
        return 'недвижимость'
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    else:
        return 'образование'

In [67]:
# применим данную функцию и создадим новый столбец
data['purpose_category'] = data.apply(get_category_purpose, axis=1)

In [68]:
# проверим, что в столбце создалась необходимая нам категория
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,children_category,total_income_category,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,клиенты с детьми,высокий уровень дохода,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,клиенты с детьми,средний уровень дохода,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,клиенты без детей,средний уровень дохода,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,клиенты с детьми,высокий уровень дохода,образование
4,0,3682,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,клиенты без детей,средний уровень дохода,свадьба


In [69]:
# выведем словарь по категории children_category
print(data['children_category'].value_counts())

клиенты без детей    14143
клиенты с детьми      7310
Name: children_category, dtype: int64


In [70]:
# выведем словарь по категории family_status
print(data['family_status'].value_counts())

женат / замужем          12339
гражданский брак          4150
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


In [71]:
# выведем словарь по категории total_income_category
print(data['total_income_category'].value_counts())

средний уровень дохода    11260
высокий уровень дохода     5306
низкий уровень дохода      4887
Name: total_income_category, dtype: int64


In [72]:
# выведем словарь по категории purpose_category
print(data['purpose_category'].value_counts())

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_category, dtype: int64


**Вывод**

Категоризация данных проводилась для четырех столбцов, для того, чтобы в следующем разделе была возможность ответить на поставленные вопросы.
Были выделены следующие словари категорий:
1. по столбцу children:
    - клиенты без детей
    - клиенты с детьми
2. по столбцу family_status:
    - женат/замужем
    - не женат/не замужем
    - гражданский брак
    - в разводе
    - вдова/вдовец
3. по столбцу total_income:
    - высокий уровень дохода
    - средний уровень дохода
    - низкий уровень дохода
4. по столбцу purpose:
    - недвижимость
    - автомбиль
    - образование
    - свадьба
    
Теперь данные полностью готовы. Проведем дальнейший анализ и ответим на поставленные вопросы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на вопрос, есть ли зависимость между наличием детей и возвратом кредита в срок, необходимо сформировать таблицу по категориям клиентов, рассчитать по этим параметрам коэффициент встречаемости непогашенных в срок долгов. Для этого нужно будет рассчитать этот коэффициент, путем деления количества не погашенных в срок кредитов на общее количество значений в выборке. Чем больше это значение, тем больше вероятность того, что клиенты из данной категории не погасят кредит в срок.

In [73]:
# создадим сводную таблицу для того, чтобы ответить на поставленный вопрос и высчитаем долю по долгам
pivot_table_children = data.pivot_table(index='children_category', columns='debt', values='children', aggfunc='count')
pivot_table_children['share'] = pivot_table_children[1] / (pivot_table_children[1] + pivot_table_children[0])

In [74]:
# выведем итоговую таблицу
display(pivot_table_children.sort_values('share', ascending=False))

debt,0,1,share
children_category,,,
клиенты с детьми,6636,674,0.092202
клиенты без детей,13076,1067,0.075444


**Вывод**

В полученной таблице 0 - нет задолженнойстей, 1 - есть задолженность, share доля клиентов, у которых есть задолженность.

По полученной таблице видно, что зависимость между наличием детей и возвратом кредита в срок существует.
Так, реже всего имеют пробломы с погашением кредита в срок клиенты, у которых нет детей. А вот клиенты с детьми чаще всего оплачивают кредит не в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Для ответа на вопрос, есть ли зависимость между семейным положением и возвратом кредита в срок, необходимо сформировать таблицу по категориям клиентов, рассчитать по этим параметрам коэффициент встречаемости непогашенных в срок долгов. Для этого нужно будет рассчитать этот коэффициент, путем деления количества непогашенных в срок кредитов на общее количество значений в выборке. Чем больше это значение, тем больше вероятность того, что клиенты из данной категории не погасят кредит в срок.

In [75]:
# создадим сводную таблицу для того, чтобы ответить на поставленный вопрос и высчитаем долю по долгам
pivot_table_family_status = data.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
pivot_table_family_status['share'] = pivot_table_family_status[1] / (pivot_table_family_status[1] + pivot_table_family_status[0])

In [76]:
# выведем итоговую таблицу
display(pivot_table_family_status.sort_values('share', ascending=False))

debt,0,1,share
family_status,,,
не женат / не замужем,2536,274,0.097509
гражданский брак,3762,388,0.093494
женат / замужем,11408,931,0.075452
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


**Вывод**

По полученной таблице видно, что зависимость между семейным статусом и возвратом кредита в срок существует. Так, реже всего не оплачивают кредиты  в срок клиенты, которые относятся к категории вдовец/вдова, чуть менее ответственны в выплате долгов - в разводе, и женат/замужем. А вот те, кто не состоят в браке чаще всего имеют просрочки по оплате кредита, так же достаточно часто имеют просрочки по опатам кредита и клиенты, которые состоят в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на вопрос, есть ли зависимость уровнем дохода и возвратом кредита в срок, необходимо сформировать таблицу по категориям клиентов, рассчитать по этим параметрам коэффициент встречаемости непогашенных долгов. Для этого нужно будет рассчитать этот коэффициент, путем деления количества просроченных кредитов на общее количество значений в выборке. Чем больше это значение, тем больше вероятность того, что клиенты из данной категории не погасят кредит в срок.

In [77]:
# определим количество непогашенных в срок кредитов, а также общее чило кредитов по категориям
data_grouped_total_income_category = data.groupby('total_income_category').agg({'debt': ['count', 'sum']})

In [78]:
# рассчитаем коэффициент встречаемости непогашенных долгов, используем конструктию try-except, т.к. можен возникнуть, например, деление на 0
try:
    data_grouped_total_income_category['friquency_debt_total_income'] = data_grouped_total_income_category['debt']['sum'] / data_grouped_total_income_category['debt']['count']
except ValueError:
    data_grouped_total_income_category['friquency_debt_total_income'] = 0

In [79]:
# выведем полученные данные
display(data_grouped_total_income_category.sort_values('friquency_debt_total_income', ascending=False))  

debt      friquency_debt_total_income
                        count  sum                            
total_income_category                                         
средний уровень дохода  11260  980                    0.087034
низкий уровень дохода    4887  384                    0.078576
высокий уровень дохода   5306  377                    0.071052

**Вывод**

По сравнению с предыдущими выводами в данном выводе невозможно однозначно сказать, влияет ли уровень дохода на наличие непогашенных кредитов. Значения не сильно отличаются друг от друга, но и по этим данным видно, что клиенты с высоким и низким уровнями дохода реже не погашают кредиты, чем клиенты со средним уровнем дохода. 

- Как разные цели кредита влияют на его возврат в срок?

Для ответа на вопрос, есть ли зависимость между целями получения кредита и возвратом кредита в срок, необходимо сформировать таблицу по категориям клиентов, рассчитать по этим параметрам коэффициент встречаемости непогашенных долгов. Для этого нужно будет рассчитать этот коэффициент, путем деления количества просроченных кредитов на общее количество значений в выборке. Чем больше это значение, тем больше вероятность того, что клиенты из данной категории не погасят кредит в срок.

In [80]:
# определим количество непогашенных в срок кредитов, а также общее число кредитов по категориям
data_grouped_purpose_category = data.groupby('purpose_category').agg({'debt': ['count', 'sum']})

In [81]:
# рассчитаем коэффициент встречаемости непогашенных долгов, используем конструктию try-except, т.к. можен возникнуть, например, деление на 0
try:
    data_grouped_purpose_category['friquency_debt_purpose'] = data_grouped_purpose_category['debt']['sum'] / data_grouped_purpose_category['debt']['count']
except ValueError:
    data_grouped_purpose_category['friquency_debt_purpose'] = 0

In [82]:
# выведем полученные данные
display(data_grouped_purpose_category.sort_values('friquency_debt_purpose', ascending=False))  

debt      friquency_debt_purpose
                  count  sum                       
purpose_category                                   
автомобиль         4306  403               0.093590
образование        4013  370               0.092200
свадьба            2323  186               0.080069
недвижимость      10811  782               0.072334

**Вывод**

На основании полученных данных можно сделать выводы о том, что существует зависимость между целями получения кредита и наличием просрочек в платеж. Так к оплате кредита более ответственно относятся люди, чьей целью получения кредита является недвижимость и свадьба. Менее ответственно к погашению кредитов и как следствие, имебщие больше всего просрочек в платежах это клиенты, цель получения которых автомобиль и образование.

## Шаг 4. Общий вывод

В ходе работы над проектом были сделано следующее:
1. Обработана таблица: удалены дубликаты, выявлены и заполнены пропуски в значениях, выявлены и исправлены аномалии в значениях, все значения приведены к одному регистру, также были изменены некоторые типы данных: вещественный на целочисленный (для того, чтобы не проблем из-за округлений данных.
2. Была проведена работа по лемматизации части данных для того, чтобы появилась возможность категоризировать некоторые данные.
3. Были выделены категории в зависимости от количества детей, семейного статуса, уровня дохода, целей получения кредита.
4. На основании категоризации мы смогли ответить на поставленные вопросы:
    - есть ли зависимость между наличием детей и возвратом кредита в срок: да, зависимость существует. Чаще всего проблемы с погашением кредита имеют клиенты с детьми, а бездетные клиенты чаще погашают кредит в срок;
    - есть ли зависимость между семейным положением и возвратом кредита в срок: да, зависимость существует. Чаще всего не погашают в срок кредиты клиенты не состоящие в браке или состоящие в гражданском браке. Более ответственно к погашению кредита относятся люди, которые сосоят в браке или наъодятся в разводе. Самим дисциплинированными оказались клиенты из категории вдовец/вдова;
    - есть ли зависимость между уровнем дохода и погашением кредита в срок: по полученным данным однозначного ответа на этот вопрос дать нельзя, так как полученные значения несильно отличаются друг от друга. Но все можно выделить, что клиенты с высоки и низким уровнями дохода ответственное погашают кредиты;
    - как разные цели кредита влияют на его возврат в срок: однозначно можно сказать, что зависимость существует. Клиенты реже имеют просрочки по кредитам, если целью кредита является: недвижимость и свадьба, и часто имеют проблемы с погашением кредита, если цель - образование и автомобиль.
    
Ответы на поставленные вопросы даны. Исследование завершено.